<a href="https://colab.research.google.com/drive/190q3Fwg2MVqke478bpBM7XoodJT6Tv36?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

To access the stable diffusion model, first connect to huggingface (where the model is stored) and login via your auth token (can be accessed by clicking on the link shown after running the cell)

In [ ]:
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


Settings for training the model

In [ ]:
MODEL_NAME = "CompVis/stable-diffusion-v1-4"
# What type of object do you want to train? The class should be something general like 'person', 'dog', 'car'
CLASS_NAME = "person"
# What is the exact name of the object you want to train the model? That string (here 'Kylie Jenner') can be then later used in prompts
INSTANCE_NAME = "Kylie Jenner"
# Where are your training images are stored? All images should be 512x512 in resolution
INSTANCE_DIR = "/content/Training_Examples"

CLASS_DIR = f"/content/data/{CLASS_NAME}"
OUTPUT_DIR = "/content/models_stable_diffusion/model_v1"
print(f"[*] Weights will be saved at {OUTPUT_DIR}")
!mkdir -p $OUTPUT_DIR

[*] Weights will be saved at /content/drive/MyDrive/Research/models_stable_diffusion/max


In [ ]:
!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v1-4" \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --instance_data_dir= $INSTANCE_DIR \
  --class_data_dir=$CLASS_DIR \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt=$INSTANCE_NAME \
  --class_prompt=$CLASS_NAME \
  --seed=5544 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --lr_scheduler="polynomial" \
  --lr_warmup_steps=400 \
  --num_class_images=8 \
  --sample_batch_size=4 \
  --gradient_accumulation_steps=8 \
  --learning_rate=3e-5 \
  --max_train_steps=2000 \
  --gradient_checkpointing

Convert the trained model into a version that can be used for generating new images

In [ ]:
ckpt_path = OUTPUT_DIR + "/model.ckpt"

half_arg = ""
# Whether to convert to fp16, takes half the space (2GB), might loose some quality.
fp16 = True
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $OUTPUT_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

Traceback (most recent call last):
  File "convert_diffusers_to_original_stable_diffusion.py", line 215, in <module>
    unet_state_dict = torch.load(unet_path, map_location="cpu")
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 699, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Research/models_stable_diffusion/max/unet/diffusion_pytorch_model.bin'
[*] Converted ckpt saved at /content/drive/MyDrive/Research/models_stable_diffusion/max2818/model.ckpt


Start up the diffusion model generator using your previously trained model.

In [ ]:
import torch
import numpy as np
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display
model_path = OUTPUT_DIR 

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")
g_cuda = torch.Generator(device='cuda')

Generate some new images using your newly trained model. Change the prompt below to use your chosen "INSTANCE_NAME" above.

In [ ]:
seed = 5124
g_cuda.manual_seed(seed)
prompt = "photo of Kylie Jenner sitting on a couch, photography, vanity fair, Nikon D810, 50 mm lens, by annie leibovitz"
num_samples = 3 # number of samples that should be generated
guidance_scale = 4 # should be a value between 1 - 15, most often choose something between 4 - 9
num_inference_steps = 120 # how many steps should be taken for image generation, try something between 30 - 150
height = 512 # size of the image
width = 512 # size of the image

with autocast("cuda"), torch.inference_mode():
  images = pipe([prompt] * num_samples, height=height, width=width, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, generator=g_cuda).images

for indx, img in enumerate(images):
  display(img)

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

...

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

## Face Restoration

Optional face restoration to use when stable diffusion outputs not realistic images of human faces

https://github.com/TencentARC/GFPGAN

In [ ]:
!git clone https://github.com/TencentARC/GFPGAN.git
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr

# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib

!pip install -r requirements.txt
!python setup.py develop

# If you want to enhance the background (non-face) regions with Real-ESRGAN,
# you also need to install the realesrgan package
!pip install realesrgan

!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
# Ausführen der Face restoration im Ordner "/content/image_output"
!python /content/GFPGAN/inference_gfpgan.py -i /content/image_output -o /content/image_output_restored_face -v 1.3 -s 2

Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth" to /usr/local/lib/python3.7/dist-packages/weights/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:02<00:00, 23.8MB/s]
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/gfpgan/weights/detection_Resnet50_Final.pth

100% 104M/104M [00:04<00:00, 24.4MB/s]
Downloadi